In [ ]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/result_subject2/'

/content/drive/MyDrive


In [ ]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import *
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [ ]:
data_dir

'/content/drive/MyDrive/data/'

In [ ]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

     |████████████████████████████████| 6.4MB 12.1MB/s 
     |████████████████████████████████| 133kB 13.5MB/s 


In [ ]:
import torch
from skorch.helper import predefined_split
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import rawData2,SEEGDataset,set_random_seeds,cuda_or_cup
from grasp.config import root_dir, tmp_dir
from grasp.myskorch import plotPrediction, MyRegressor


In [ ]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import rawData2,SEEGDataset

In [ ]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [ ]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [ ]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   True


In [ ]:
%%capture
# suppress the output
sid=2
sampling_rate=1000
traindata, valdata, testdata = rawData2(2,'band','all',move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)


In [ ]:
trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]

samples=trainx.shape[0]
chnNum=trainx.shape[1]
step=50 #ms
T=1000 #ms
totalLen=trainx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280

train_ds = SEEGDataset(trainx, trainy,T, step)
val_ds = SEEGDataset(valx, valy,T, step)
test_ds = SEEGDataset(testx, testy,T, step)


In [ ]:
learning_rate=0.001
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
dropout=0.5
#Lambda = 1e-6
Lambda = 0
# Do not init here!!
model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()
 
#checkpoint = torch.load(result_dir+'checkpoint195.pth')
#net.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
from skorch.callbacks import Checkpoint, TrainEndCheckpoint
cp = Checkpoint(dirname=result_dir) # Will automatically save the model when there is an improvement during validation.
train_end_cp = TrainEndCheckpoint(dirname=result_dir) # save model and the end of the training

In [ ]:

net = MyRegressor(
    model,
    iterator_train__shuffle=True,
    train_split=predefined_split(val_ds),
    max_epochs=200,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)),cp,train_end_cp ],
    device = device
)


In [ ]:
#new_net.initialize() # This is important!
#new_net.load_params(f_params=result_dir+'model.pkl', f_optimizer=result_dir+'opt.pkl', f_history=result_dir+'history.json')

In [ ]:
net.fit(train_ds,y=None)

In [ ]:
net.save_params(f_params=result_dir+'model.pkl', f_optimizer=result_dir+'opt.pkl', f_history=result_dir+'history.json')

In [ ]:
# continue training....
from skorch.callbacks import LoadInitState
load_state = LoadInitState(cp)
#load_state = LoadInitState(train_end_cp)

net = MyRegressor(
    model,
    #train_split=predefined_split(valid_set),
    iterator_train__shuffle=True,
    train_split=predefined_split(val_ds),
    max_epochs=200,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)), cp, train_end_cp, load_state],
    device = device
)

net.fit(train_ds,y=None)